Train an imitator using the best hyperparameters from multiple optuna studies. The Hyperparameters were chosen using time series splits, while this agent will be trained on the entire dataset.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, Subset

import pandas as pd
import os

In [2]:
BATCH = 2**6
INPUTS = 31
CLASSES = 20
DEVICE = 'cuda'
EPOCHS = 1000
SEED = 42
DATADIR = 'PPO agent 100 alts over 1000+200 2-3-21 results' +'/'
DATASET = 'baseline_obs-a.csv'
SAVEDIR = DATADIR + 'surrogates' + '/'
NAME = 'imitator'

In [3]:
class MyDataset(Dataset):
    def __init__(self, df):
        self.data = torch.tensor(df.iloc[:, :-1].values, dtype=torch.float32)
        self.targets = torch.tensor(df.iloc[:, -1].values, dtype=torch.long)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]

In [4]:
def load_SA_dataset():
    #path to data directory
    data_dir = os.path.join(os.getcwd(), DATADIR) #run this from citylearn folder
    data_dir = os.path.normpath(data_dir) #resolve '..'
    #load features
    df_obs = pd.read_csv(data_dir + '/rebaseline obs.csv',
                    header=None,
                    dtype='float32')
    df_obs.set_index(df_obs.index.astype(int), inplace=True)
    #load labels
    df_a = pd.read_csv(data_dir + '/rebaseline a.csv',
                header=None,
                dtype='float32')
    df_a.set_index(df_a.index.astype(int), inplace=True)
    df_obs['a'] = df_a[0]
    return df_obs

In [5]:
def get_data_subset(dataset,train_ids, valid_ids, batch_size):
    """returns dataloaders with no shuffling as timeseries samples are NOT iid"""
    train_subsampler = Subset(dataset, 
                              train_ids)
    valid_subsampler = Subset(dataset, 
                              valid_ids)
    train_loader = torch.utils.data.DataLoader(train_subsampler,
                                              batch_size=batch_size,)
    valid_loader = torch.utils.data.DataLoader(valid_subsampler, 
                                             batch_size=batch_size,)
    return train_loader, valid_loader

Load training data

In [6]:
df_SA = pd.read_csv(DATADIR+DATASET,
                    index_col=0)#load_SA_dataset()
sa_dataset = MyDataset(df=df_SA)
data_loader = torch.utils.data.DataLoader(sa_dataset,
                                          batch_size=BATCH,)

Def model

In [7]:
model = nn.Sequential(
    nn.Linear(INPUTS, 300),
    nn.Tanh(),
    nn.Dropout(0.388),
    nn.Linear(300,493),
    nn.Tanh(),
    nn.Dropout(0.145),
    nn.Linear(493, CLASSES)
).to(DEVICE)

In [8]:
optimizer = optim.RMSprop(model.parameters(),
                          lr=3.3e-4)

Set random seed for reproducability

In [9]:
torch.manual_seed = SEED

Train and evaluation on entire dataset

In [10]:
for epoch in range(EPOCHS):
                model.train()
                for batch_idx, (data, target) in enumerate(data_loader):

                    data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)

                    optimizer.zero_grad()
                    output = model(data)
                    loss = F.cross_entropy(output, target)
                    loss.backward()
                    optimizer.step()

                # Validation of the model.
                model.eval()
                correct = 0
                with torch.no_grad():
                    for batch_idx, (data, target) in enumerate(data_loader):
                        data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)
                        output = model(data)
                        # Get the index of the max logit
                        pred = output.argmax(dim=1, keepdim=True)
                        correct += pred.eq(target.view_as(pred)).sum().item()

                acc_epoch = correct / len(data_loader.dataset)
                print(f'Epoch: {epoch}, Acc: {acc_epoch}')

Epoch: 0, Acc: 0.419340107318187
Epoch: 1, Acc: 0.4354378353693344
Epoch: 2, Acc: 0.45050804886402557
Epoch: 3, Acc: 0.4648932526544126
Epoch: 4, Acc: 0.4836168512387259
Epoch: 5, Acc: 0.496860372188606
Epoch: 6, Acc: 0.5080488640255737
Epoch: 7, Acc: 0.5168398218974769
Epoch: 8, Acc: 0.5216348898276059
Epoch: 9, Acc: 0.5280283137344446
Epoch: 10, Acc: 0.5303116794154584
Epoch: 11, Acc: 0.5355634204817902
Epoch: 12, Acc: 0.5408151615481219
Epoch: 13, Acc: 0.5440118735015412
Epoch: 14, Acc: 0.5498344559881265
Epoch: 15, Acc: 0.5532595045096472
Epoch: 16, Acc: 0.5524603265212924
Epoch: 17, Acc: 0.5564562164630665
Epoch: 18, Acc: 0.5572553944514214
Epoch: 19, Acc: 0.5627354720858545
Epoch: 20, Acc: 0.5670738668797808
Epoch: 21, Acc: 0.5655896791871218
Epoch: 22, Acc: 0.5704989154013015
Epoch: 23, Acc: 0.5735814590706702
Epoch: 24, Acc: 0.5790615367051033
Epoch: 25, Acc: 0.5828290900787761
Epoch: 26, Acc: 0.5833999314990296
Epoch: 27, Acc: 0.5884233359972599
Epoch: 28, Acc: 0.5908208699623

In [11]:
torch.save(model, SAVEDIR + NAME + '.pth')